# 说明
这是一篇纯粹的例子。

作用是爬取百度贴吧的某个帖子的图片:
<img src='./image/tieba.png' />

In [16]:
from bs4 import BeautifulSoup
import requests
import contextlib
import sys
import os
import traceback
from tqdm import tqdm

In [17]:
headers = {'Connection': "keep-alive",
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'zh-CN,zh;q=0.9'}

In [18]:
def Schedule(a, b, c):
    per = 100.0 * a * b / c
    if per > 100:
        per = 100
    print("  " + "%.2f%% 已经下载的大小:%ld 文件大小:%ld" % (per, a * b, c) + '\r')

In [4]:
def urlretrieve(url, filename=None, reporthook=None, params=None):
    print('download file: {0}'.format(url))
    '''传入ID改变url，利用closing跟iter_content下载图片'''
    with contextlib.closing(requests.get(url, stream=True, headers=headers, params=params)) as fp:  # 打开网页
        header = fp.headers
        with open(filename, 'wb+') as tfp:  # w是覆盖原文件，a是追加写入 #打开文件
            bs = 1024
            size = -1
            blocknum = 0
            if "content-length" in header:
                size = int(header["Content-Length"])  # 文件的总大小理论值
            if reporthook and size > 0:
                reporthook(blocknum, bs, size)  # 写入前运行一次回调函数

            for chunk in fp.iter_content(chunk_size=1024):
                if chunk:
                    tfp.write(chunk)  # 写入
                    tfp.flush()
                    blocknum += 1
                    if reporthook and size > 0:
                        reporthook(blocknum, bs, size)  # 每写入一次就运行一次回调函数

In [14]:
def crawl(url, limits: int = None):
    try:
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        imglist = soup.find_all('img', class_='BDE_Image')
        folder = './crawlfile/tieba'
        if limits is not None and limits > 0:
            imglist = imglist[0:limits]
        for index, imgtag in enumerate(tqdm(imglist)):
            print('download {0}/{1} files'.format(index+1, len(imglist)))
            imgurl = str(imgtag['src'])
            print(imgurl)
            if '.jpg' in imgurl:
                urlretrieve(url=imgurl,
                            filename=os.path.join(folder,
                                                  '{0}.jpg'.format(index+1)), reporthook=Schedule)


        print(r.encoding)
    except Exception as e:
        print(e)
        traceback.print_exc()

In [20]:
soup = BeautifulSoup('<html></html>', 'html.parser')
help(soup.find_all)

Help on method find_all in module bs4.element:

find_all(name=None, attrs={}, recursive=True, string=None, limit=None, **kwargs) method of bs4.BeautifulSoup instance
    Look in the children of this PageElement and find all
    PageElements that match the given criteria.
    
    All find_* methods take a common set of arguments. See the online
    documentation for detailed explanations.
    
    :param name: A filter on tag name.
    :param attrs: A dictionary of filters on attribute values.
    :param recursive: If this is True, find_all() will perform a
        recursive search of this PageElement's children. Otherwise,
        only the direct children will be considered.
    :param limit: Stop looking after finding this many results.
    :kwargs: A dictionary of filters on attribute values.
    :return: A ResultSet of PageElements.
    :rtype: bs4.element.ResultSet



In [15]:
folder = './crawlfile/tieba'
if not os.path.exists(folder):
    os.makedirs(folder)
# 之前的帖子没了，换这个帖子
crawl('https://tieba.baidu.com/p/7209567277', limits=10)

  0%|                                                                                      | 0/10 [00:00<?, ?it/s]

download 1/10 files
http://tiebapic.baidu.com/forum/w%3D580/sign=aa1230c9dc1349547e1ee86c664f92dd/e3735fdf8db1cb13bd99b45bca54564e90584be6.jpg?tbpicau=2022-11-18-05_837c947df419fcc7501560b2a790b1bb
download file: http://tiebapic.baidu.com/forum/w%3D580/sign=aa1230c9dc1349547e1ee86c664f92dd/e3735fdf8db1cb13bd99b45bca54564e90584be6.jpg?tbpicau=2022-11-18-05_837c947df419fcc7501560b2a790b1bb
  0.00% 已经下载的大小:0 文件大小:109536
  0.93% 已经下载的大小:1024 文件大小:109536
  1.87% 已经下载的大小:2048 文件大小:109536
  2.80% 已经下载的大小:3072 文件大小:109536
  3.74% 已经下载的大小:4096 文件大小:109536
  4.67% 已经下载的大小:5120 文件大小:109536
  5.61% 已经下载的大小:6144 文件大小:109536
  6.54% 已经下载的大小:7168 文件大小:109536
  7.48% 已经下载的大小:8192 文件大小:109536
  8.41% 已经下载的大小:9216 文件大小:109536
  9.35% 已经下载的大小:10240 文件大小:109536
  10.28% 已经下载的大小:11264 文件大小:109536
  11.22% 已经下载的大小:12288 文件大小:109536
  12.15% 已经下载的大小:13312 文件大小:109536
  13.09% 已经下载的大小:14336 文件大小:109536
  14.02% 已经下载的大小:15360 文件大小:109536
  14.96% 已经下载的大小:16384 文件大小:109536
  15.89% 已经下载的大小:17408 文件大小:109536
  1

 10%|███████▊                                                                      | 1/10 [00:00<00:02,  3.99it/s]

  49.55% 已经下载的大小:54272 文件大小:109536
  50.48% 已经下载的大小:55296 文件大小:109536
  51.42% 已经下载的大小:56320 文件大小:109536
  52.35% 已经下载的大小:57344 文件大小:109536
  53.29% 已经下载的大小:58368 文件大小:109536
  54.22% 已经下载的大小:59392 文件大小:109536
  55.16% 已经下载的大小:60416 文件大小:109536
  56.09% 已经下载的大小:61440 文件大小:109536
  57.03% 已经下载的大小:62464 文件大小:109536
  57.96% 已经下载的大小:63488 文件大小:109536
  58.90% 已经下载的大小:64512 文件大小:109536
  59.83% 已经下载的大小:65536 文件大小:109536
  60.77% 已经下载的大小:66560 文件大小:109536
  61.70% 已经下载的大小:67584 文件大小:109536
  62.64% 已经下载的大小:68608 文件大小:109536
  63.57% 已经下载的大小:69632 文件大小:109536
  64.50% 已经下载的大小:70656 文件大小:109536
  65.44% 已经下载的大小:71680 文件大小:109536
  66.37% 已经下载的大小:72704 文件大小:109536
  67.31% 已经下载的大小:73728 文件大小:109536
  68.24% 已经下载的大小:74752 文件大小:109536
  69.18% 已经下载的大小:75776 文件大小:109536
  70.11% 已经下载的大小:76800 文件大小:109536
  71.05% 已经下载的大小:77824 文件大小:109536
  71.98% 已经下载的大小:78848 文件大小:109536
  72.92% 已经下载的大小:79872 文件大小:109536
  73.85% 已经下载的大小:80896 文件大小:109536
  74.79% 已经下载的大小:81920 文件大小:109536
  75.72% 已经下载的大小:829

 30%|███████████████████████▍                                                      | 3/10 [00:00<00:01,  4.87it/s]

...

  14.23% 已经下载的大小:48128 文件大小:338283
  14.53% 已经下载的大小:49152 文件大小:338283
  14.83% 已经下载的大小:50176 文件大小:338283
  15.14% 已经下载的大小:51200 文件大小:338283
  15.44% 已经下载的大小:52224 文件大小:338283
  15.74% 已经下载的大小:53248 文件大小:338283
  16.04% 已经下载的大小:54272 文件大小:338283
  16.35% 已经下载的大小:55296 文件大小:338283
  16.65% 已经下载的大小:56320 文件大小:338283
  16.95% 已经下载的大小:57344 文件大小:338283
  17.25% 已经下载的大小:58368 文件大小:338283
  17.56% 已经下载的大小:59392 文件大小:338283
  17.86% 已经下载的大小:60416 文件大小:338283
  18.16% 已经下载的大小:61440 文件大小:338283
  18.47% 已经下载的大小:62464 文件大小:338283
  18.77% 已经下载的大小:63488 文件大小:338283
  19.07% 已经下载的大小:64512 文件大小:338283
  19.37% 已经下载的大小:65536 文件大小:338283
  19.68% 已经下载的大小:66560 文件大小:338283
  19.98% 已经下载的大小:67584 文件大小:338283
  20.28% 已经下载的大小:68608 文件大小:338283
  20.58% 已经下载的大小:69632 文件大小:338283
  20.89% 已经下载的大小:70656 文件大小:338283
  21.19% 已经下载的大小:71680 文件大小:338283
  21.49% 已经下载的大小:72704 文件大小:338283
  21.79% 已经下载的大小:73728 文件大小:338283
  22.10% 已经下载的大小:74752 文件大小:338283
  22.40% 已经下载的大小:75776 文件大小:338283
  22.70% 已经下载的大小:768

 90%|██████████████████████████████████████████████████████████████████████▏       | 9/10 [00:02<00:00,  4.06it/s]

  0.00% 已经下载的大小:0 文件大小:291485
  0.35% 已经下载的大小:1024 文件大小:291485
  0.70% 已经下载的大小:2048 文件大小:291485
  1.05% 已经下载的大小:3072 文件大小:291485
  1.41% 已经下载的大小:4096 文件大小:291485
  1.76% 已经下载的大小:5120 文件大小:291485
  2.11% 已经下载的大小:6144 文件大小:291485
  2.46% 已经下载的大小:7168 文件大小:291485
  2.81% 已经下载的大小:8192 文件大小:291485
  3.16% 已经下载的大小:9216 文件大小:291485
  3.51% 已经下载的大小:10240 文件大小:291485
  3.86% 已经下载的大小:11264 文件大小:291485
  4.22% 已经下载的大小:12288 文件大小:291485
  4.57% 已经下载的大小:13312 文件大小:291485
  4.92% 已经下载的大小:14336 文件大小:291485
  5.27% 已经下载的大小:15360 文件大小:291485
  5.62% 已经下载的大小:16384 文件大小:291485
  5.97% 已经下载的大小:17408 文件大小:291485
  6.32% 已经下载的大小:18432 文件大小:291485
  6.67% 已经下载的大小:19456 文件大小:291485
  7.03% 已经下载的大小:20480 文件大小:291485
  7.38% 已经下载的大小:21504 文件大小:291485
  7.73% 已经下载的大小:22528 文件大小:291485
  8.08% 已经下载的大小:23552 文件大小:291485
  8.43% 已经下载的大小:24576 文件大小:291485
  8.78% 已经下载的大小:25600 文件大小:291485
  9.13% 已经下载的大小:26624 文件大小:291485
  9.49% 已经下载的大小:27648 文件大小:291485
  9.84% 已经下载的大小:28672 文件大小:291485
  10.19% 已经下载的大小:29696 文件大小

100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.31it/s]

  0.00% 已经下载的大小:0 文件大小:348498
  0.29% 已经下载的大小:1024 文件大小:348498
  0.59% 已经下载的大小:2048 文件大小:348498
  0.88% 已经下载的大小:3072 文件大小:348498
  1.18% 已经下载的大小:4096 文件大小:348498
  1.47% 已经下载的大小:5120 文件大小:348498
  1.76% 已经下载的大小:6144 文件大小:348498
  2.06% 已经下载的大小:7168 文件大小:348498
  2.35% 已经下载的大小:8192 文件大小:348498
  2.64% 已经下载的大小:9216 文件大小:348498
  2.94% 已经下载的大小:10240 文件大小:348498
  3.23% 已经下载的大小:11264 文件大小:348498
  3.53% 已经下载的大小:12288 文件大小:348498
  3.82% 已经下载的大小:13312 文件大小:348498
  4.11% 已经下载的大小:14336 文件大小:348498
  4.41% 已经下载的大小:15360 文件大小:348498
  4.70% 已经下载的大小:16384 文件大小:348498
  5.00% 已经下载的大小:17408 文件大小:348498
  5.29% 已经下载的大小:18432 文件大小:348498
  5.58% 已经下载的大小:19456 文件大小:348498
  5.88% 已经下载的大小:20480 文件大小:348498
  6.17% 已经下载的大小:21504 文件大小:348498
  6.46% 已经下载的大小:22528 文件大小:348498
  6.76% 已经下载的大小:23552 文件大小:348498
  7.05% 已经下载的大小:24576 文件大小:348498
  7.35% 已经下载的大小:25600 文件大小:348498
  7.64% 已经下载的大小:26624 文件大小:348498
  7.93% 已经下载的大小:27648 文件大小:348498
  8.23% 已经下载的大小:28672 文件大小:348498
  8.52% 已经下载的大小:29696 文件大小: